# Introducción a las redes RNN-LSTM

## Contenido
### 1. Instalando 'yfinance' y usándolo para importar datos del Bitcoin (utilizando Ticker y get_data_yahoo).
### 2. Prueba para importar múltiples datos (opcional).
### 3. Creación y explicación de la función "univariate_data()" para la preparación de datos.

### *Importando paquetes:*

In [ ]:
#RNN
import tensorflow as tf
#Graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
#Setting figure size
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False

### *Instalando yfinance:*

In [ ]:
#installing yfinance (Colab)
!pip install yfinance
#pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3MB 10.9MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=9a734ab44971917a1927109f461fc00d75732a26d6896a8a6a88c8d524c0cc87
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


### *Importando y visualizando datos del bitcoin:*


In [ ]:
import yfinance as yf

from pandas_datareader import data as pdr

yf.pdr_override() # <== that's all it takes :-)

#Univariate time series
#One way - get_data_yahoo:
btc_usd = pdr.get_data_yahoo("BTC-USD")
print(type(btc_usd))
btc_usd.head()
#btc_usd.sort_index(ascending=False).head()

[*********************100%***********************]  1 of 1 completed
<class 'pandas.core.frame.DataFrame'>


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


## (Opcional):
La siguiente celda se puede omitir pues solo muestra la aplicación de unos métodos para series de tiempo univariadas (usando Ticker) y como importar series de tiempo multivariadas:

In [ ]:
###NOTE: minimum interval: 1 minute
#------------------------
#Univariate time series
#------------------------
btc_usd_T = yf.Ticker("BTC-USD")
#type(btc_usd)
#Exploring some methods
#btc_usd.info
#btc_usd.actions
#btc_usd.financials
#btc_usd.history(period="max")
#btc_usd.calendar
#### valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
#btc_usd.history(period="2d",interval="1m") 

#-------------------------
#Multivariate time series
#-------------------------
starDate="2017-01-01" #year-month-day
endDate="2017-04-30"
data = pdr.get_data_yahoo("BTC-USD ETH-USD", start=starDate, end=endDate)
#print(data)
#print(data["Adj Close"])
#print(type(data["Adj Close"]["BTC-USD"]))
#print(data["Adj Close"]["BTC-USD"])
dataTickers = yf.Tickers('BTC-USD ETH-USD')
#print(dataTickers.tickers["BTC-USD"].info)
#print(dataTickers.tickers["ETH-USD"].history(period="1mo"))

[*********************100%***********************]  2 of 2 completed


## Registros diarios (uno por día)
Como se puede ver hay se tienen 2420 registros diarios del Bitcoin (desde el 2014 hasta la fecha):

In [ ]:
btc_usd.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2422.000000,2422.000000,2422.000000,2422.000000,2422.000000,2.422000e+03
mean,7222.063690,7424.491231,7012.698308,7245.055123,7245.055123,1.206789e+10
std,11075.435681,11418.615660,10711.291406,11121.986873,11121.986873,1.902722e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,457.029488,462.521988,453.033508,457.392509,457.392509,6.819998e+07
50%,4167.494873,4309.098633,4047.088013,4168.899902,4168.899902,3.899710e+09
75%,9083.258789,9274.412109,8843.286865,9111.083252,9111.083252,1.808117e+10
max,63523.753906,64863.097656,62208.964844,63503.457031,63503.457031,3.509679e+11


## Función de preparación de datos univariados de entrenamiento (*candidata a exportar*)

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = [] #CHANGE FOR: history
  labels = [] #CHANGE FOR: targets

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    #CHANGE FOR: window_time
    indices = range(i-history_size,i) #it balances the start_index = start_index + history_size made above
    #print(indices,i)
    #print(indices)
    #Reshape data from (history_size,) to (history_size,1)
    data.append(np.reshape(dataset[indices], (history_size, 1))) #len(dataset[indices]) = history_size
    #print(data)
    labels.append(dataset[i+target_size-1])
  return np.array(data), np.array(labels)


#IN MY OPINION: TRAIN_SPLIT = LEN(DATASET)*0.8


Mirando como trabaja la función univariate_data:

In [ ]:
test = np.arange(0,10)
#print(type(test))
#print(test)
print(test[range(0,4)]) #equivalent to dataset[indices]
print(np.reshape(test[range(0,4)], (4,1)))
print(test.shape[])
x,y = univariate_data(dataset=test, start_index=0, end_index=8, history_size=4,target_size=1)
#print(x)
#print(y)
#a = range(-5,5)

[0 1 2 3]
[[0]
 [1]
 [2]
 [3]]
(10,)


## Bibliografía
### Redes RNN-LSTM (cuaderno).
1. [Introducción a Keras LSTM para series de tiempo multivariadas. Montenegro et.al.](https://nbviewer.jupyter.org/github/AprendizajeProfundo/Diplomado/blob/master/Temas/M%C3%B3dulo%205-%20Redes%20recurrentes/Cuadernos/rnrTimes_series_Multivariate.ipynb).
1. 1. [Notebook community: Time series forecasting (Ori. English)](https://notebook.community/tensorflow/docs-l10n/site/en-snapshot/tutorials/structured_data/time_series).
1. 2. [Keras Timeseries Multi-Step Multi-Output](https://www.kaggle.com/nicapotato/keras-timeseries-multi-step-multi-output).
### Yahoo Finance (datos importados).
2. [Yahoo Finance API – A Complete Guide](https://algotrading101.com/learn/yahoo-finance-api-guide/)
3. [yfinance Library – A Complete Guide](https://algotrading101.com/learn/yfinance-guide/)
4. [yfinance documentation](https://pypi.org/project/yfinance/)
### Cuadernos de referencia.
5. [Predicción de acciones usando análisis multivariado (repo)
](https://github.com/ninja3697/Stocks-Price-Prediction-using-Multivariate-Analysis). 
6. [Serie de tiempo multivariadas para predicción de precios en Python](https://www.relataly.com/stock-market-prediction-with-multivariate-time-series-in-python/1815/). 
7. [Conversión de datos (clima)](https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/).
8. [Predicción en series de tiempo multivariadas (clima)](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/).
### Libro
9. [François Chollet - Deep Learning with Python](https://b-ok.lat/s/Deep%20Learning%20with%20Python%20Fran%C3%A7ois%20Chollet).
### Artículo
10. [Stock Price Prediction Using CONVOLUTIONAL Neural Networks on a Multivariate Timeseries. Article](https://www.researchgate.net/publication/338477393_Stock_Price_Prediction_Using_Convolutional_Neural_Networks_on_a_Multivariate_Timeseries). 
[]().


##Links sobre criptomonedas (bitcoin especialmente)
https://unperiodico.unal.edu.co/pages/detail/criptomonedas-una-herramienta-riesgosa-pero-de-uso-inminente/

https://repositorio.udesa.edu.ar/jspui/bitstream/10908/16022/1/%5BP%5D%5BW%5D%20T.%20M.%20AyPP.%20Lojo%20M%C3%A1rquez%2C%20Tamara%20In%C3%A9s.pdf  (cap2 c, cap3 ii, cap4)

https://www.daviescoin.io/es/blog/que-determina-el-valor-de-las-criptomonedas

https://www.tekcrispy.com/2018/05/16/que-determina-precio-criptomonedas/

https://repositorio.uniagustiniana.edu.co/bitstream/handle/123456789/845/HurtadoCuellar-AngieDaniela-2019.pdf.pdf?sequence=1 (cap7)
